# 分析街道拓撲結構於探討城市交通擁塞問題

第八屆發展研究年會 2016 - C3 空間資料與永續發展

星期日, 10月16日, 2016 / 09:10-10:50(2) / 國立臺灣大學地理環境資源學系-202室


<small><b>陳威全</b> (國立臺灣大學 - 地理環境資源學系)</small>

## 交通擁塞的三個面向
- 連接性 (connectivity): 這一段路可以更快的通向哪些地方？
- 需求 (demand): 有多少車子(需要)經過這路段？
- 設計 (design): 這區域的路段網絡結構是否太複雜？

這研究的目的是想找出影響  
<big>交通擁塞</big>
的 
<big>網絡特性</big>.  
<br>
### 分析街道拓撲結構 -- 探討城市交通擁塞問題

有兩個部分:
### 移動的車子，及相互連接的路網
請想象一下，
有很多的<b>道路路段</b>，他們彼此相互連在一起，  
並且有很多<b>車子</b>在這上面移動 ...  
<div align="center">
<img src="fig/streets_01.png" alt="streets and cars" width="600"><br>
</div>
  
  
- 移動者與路網之間的關係，其實與網際網路中的瀏覽者與 WWW 之間的結構很相似

#### 從簡單的模型開始思考
  
以一個簡單的路網結構爲例:  
- 當一輛車子走在路上的時候，會沿着道路走...  
- 當車子走到交叉路口，可能會往前走、可能會左轉、可能會右轉...  

一開始,  
先假設車子左轉、右轉、直走的機率是一樣的。 

<div align="center"><img src="fig/car.gif" alt="random turn" width="700"></div><br>

<div align="center">
<sup><sub>simulating car movements with random turns</sub></sup>
<img src="fig/random_turn.png" alt="random turn" width="400"><br>
轉向至每一個相連的路段的機率都 = $ 1/3 $</div>
<br>

將這個模型延伸到臺北市路網

In [3]:
from IPython.display import HTML
random_move = HTML('<iframe src="https://player.vimeo.com/video/168713881" width="640"\
height="360" frameborder="0" webkitallowfullscreen mozallowfullscreen \
allowfullscreen></iframe><p><a href="https://vimeo.com/168713881">\
simulation - moving cars on streets taking turn randomly - v0.2</a>\
from <a href="https://vimeo.com/user11082431">Benny Chin</a> on \
<a href="https://vimeo.com">Vimeo</a>.</p>')

In [4]:
random_move

這是一個隨機移動模型(random walk model)：
<b>人會在一個空間中依循道路隨意的移動的模型</b>。

隨機移動模型在一定的條件下，最後停留在每一個地點的人數會逐漸進入穩定的狀態。
- 條件1：可能經過的路段是固定的
- 條件2：每一路段當下的車輛數(t)，只會受到其相連路段的前一時刻(t-1)的車輛數所影響
- 條件3：路段間的轉移機率是固定的

如果隨機移動的過程符合上述條件，透過一些隨機移動模型的計算方法，
則可以算出在這樣的移動特性下，每一段路段上的流量(停留)的多寡。

PageRank 基本形態 (Brin & Page, 1998)  

$$ PR_{t}(i) = \sum_{j\in IN(i)} PR_{t-1}(j) \times \frac{1}{outdeg(j)} $$  
$$ IN(i) : 指向 i 節點的鄰居; $$ 
$$ outdeg(j): j 節點的指出數. $$

<div align="center">
PageRank (基本形態) 的示意圖
<img src="fig/PR_demo.png" alt="webpage network" width="400"><br>
選擇每一個(相連)的下一個目標的機率都 $ =  1/3 $</div>
<br>

所以，  
我們用 PageRank 分析街道路網，  
以偵測出關鍵的路段 --車子都會經過-- 的路段。  

PageRank 的結果可以反映街道的「關鍵程度」。  
其實際意義反映的是

### 潛在的(流過) 需求量

### 但是,
真實世界中，  
車子的流動不是完全的隨機移動。  


<div align="center">
<img src="fig/prefered_turn.png" alt="main street vs. secondary street" width="400"><br>
比方說， 主要幹道 vs. 小巷道</div>
<br>

基本上大多數的車子會往「高引力」的方向走.

爲了可以捕捉這種移動方式,  
我們加入了一個路段特性的概念 -- 吸引程度 ($attr$) -- 來反映目的地路段之間的差異狀況。 

$ attr $ 的物理意義是，目的地路段吸引車子流動過去的能力

透過這樣的一個特性，進一步調整路段之間轉向機率的計算方式。  

轉向機率:  

$$ turn(j,i) = \frac{attr(i)}{\sum_{k\in OUT(j)}attr(k)} $$

<div align="center">
<img src="fig/prefered_turn.png" alt="main street vs. secondary street" width="400"><br></div>
<br>

<div align="center">
$ attr(A) = 2 $  
$ attr(B) = 5 $  
$ attr(C) = 3 $  
$ turn(O,B) = \frac{5}{2+3+5} = 0.5$</div>

但，我們如何知道影響路段吸引程度的空間分佈狀況呢？  
- 應用部分已知路段的資料，透過「基因演算法」調校所有路段的吸引程度。
- GA 的配適度： FBPR 運算結果的已知路段部分與已知路段的流速之間的 Spearman's rank correlation

修改的 PR: Flow-based PageRank (FBPR):  

$$ FBPR_{t}(i) = \sum_{j\in IN(i)} FBPR_{t-1}(j) \times turn(j,i) $$  

<div align="center"><img src="fig/car2.gif" alt="random turn" width="700"></div><br>

吸引程度可以透過 GA 調校，  
然後，轉向機率就可以透過吸引程度計算出來  
  
並且，我們就可以進一步計算 
#### 路段的區域複雜度 (local complexity). 

$$ entropy(j) = -\sum_{k \in OUT(j)} (turn(j,k) \times ln(turn(j,k))) $$
#### 轉出複雜度 (outgoing entropy)
<small>從該路段轉出去的車子是否都走向同一個方向，  
或是平均的走向所有可轉出的目的地.</small>

<div align="center">
主導的轉出 vs. 均勻的轉出
<img src="fig/entropy.png" alt="entropy" width="1020"><br>
轉出複雜度 -- 區域的複雜度</div>
<br>

小結,  
- 有 2 個部分: <b>移動的車子</b> 及 <b>路網</b>
- <b>轉向機率</b> 路段之間移動的機率
- FBPR 分數分佈 可捕捉 <b>潛在的需求</b> 分佈
- 轉出複雜度可反映路段的 <b>區域的複雜度</b> 

#### 關於擁塞的三個概念
- 連接性: 對街道路網執行修改的 PR 演算法(FBPR)，這是一個捕捉隨機移動者模式的演算法
- 潛在需求: FBPR scores 的分佈
- 區域複雜度: 轉出複雜度的分佈


### 這研究的目的： 從街道的結構特性與現有的流量資料，分析容易形成擁塞的區域路段

# 交通擁塞問題
是世界各大城市幾乎每天都會發生的問題.  
  
在很多發展中國家，路網的承載能力的提升速度追不上車子擁有狀況的增加速度，導致逐漸嚴重的擁塞狀況。

<div align="center">
<small>擁塞: 是一個時空問題</small>
<img src="fig/00_googletraffic.png" alt="google traffic" width="1024"><br>
臺北市</div>
<br>

<small>(除了不預期的事件,如 交通事故)</small>  
交通擁塞會發生，是因爲  
大多數的人試着在同樣的時間，  
前往同樣的目的地 (工作地點、學校)，  
造就了 對街道路網 上很高的壓力。  
<br>
<br>
尖峯時間:
- 需求上升
- 複雜的街道容易形成擁塞

# 架構

1. 應用<b>路網結構</b>與部分的<b>流量資料</b> 結合 <b>FBPR</b>，  
2. 計算整體路網的<b>潛在流量分佈</b>，  
3. 過程中調校出<b>轉向機率</b>的分佈，  
    進一步計算成<b>轉出複雜度</b>，  
5. 整合潛在流量分佈與轉出複雜度分佈，  
    偵測出<b>容易擁塞的區段</b>。

<div align="center">
<img src="fig/04_studyframework.png" alt="study framework" width="800"><br>
研究流程</div>
<br>

<div align="center">
<img src="fig/01_study_area.png" alt="something about Taipei" width="800"><br>
臺北市 </div>
<br>

### the connectivity
the ability of streets for bringing people to their destination.


- analysis unit (nodes): street segment
- connection (links): intersection of street segments
- network: the dual graph of street network

<div align="center">
<img src="fig/dual_representing.png" alt="dual representation" width="800"><br>
from street network to its dual graph</div>
<br>

### the (potential) demands
the potential number of vehicles passing by the street?

the vehicle detector (VD) data: the real traffic volume data
- a sensor under the street, that count the number of vehicles that is passing above it. 
- located at most of the major streets in Taipei City

<div align="center">
<img src="fig/03_streetvd.png" alt="VD location and study area" width="400"><br>
the street network, and VD location</div>
<br>

### the design
is the network structure too complex?

- moving speed will be slowed down  
  if more vehicles are turning to different  
  direction from a same street.
- the complexity of turning probability  
  between streets is a key measurement  
  to understand traffic congestion

# 結果


## 臺北市的 FBPR 移動模式 -- 模擬狀況

In [2]:
from IPython.display import HTML
FBPR_movement = HTML('<iframe src="https://player.vimeo.com/video/168714357" width="640"\
height="360" frameborder="0" webkitallowfullscreen mozallowfullscreen \
allowfullscreen></iframe><p><a href="https://vimeo.com/168714357">\
simulation - moving cars on streets turning with the estimated probability\
- v0.2</a> from <a href="https://vimeo.com/user11082431">Benny Chin</a>\
on <a href="https://vimeo.com">Vimeo</a>.</p>')

In [3]:
FBPR_movement

## 潛在流量需求(FBPR) 與 
## 轉出複雜度 (outgoing entropy) 
的空間分佈

<div align="center">
<img src="fig/06_fbpr.png" alt="FBPR distribution" width="800"><br>
潛在流量需求(FBPR) 空間分佈</div>
<br>

<div align="center">
<img src="fig/07_entropy.png" alt="outgoing entropy distribution" width="800"><br>
轉出複雜度 (outgoing entropy) 空間分佈</div>
<br>

## 容易擁塞的路段與區域 vs. 官方擁塞路段

<div align="center">
<img src="fig/08_congested.png" alt="congested segment" width="800"><br>
容易擁塞路段： 高潛在需求與高轉出複雜度的重疊路段</div>
<br>

# 討論

## 從一個簡單的模型，到一個擁塞模型
- 2 個部分: 移動的車子 & 相互連接的路段
- 對隨機移動模型加入了「吸引程度」的概念
- 計算了轉向機率
- 估計了所有路段的潛在需求
- 量測了轉出複雜度
- 建立了 FBPR 演算法

這研究提出來的架構，  
是基於<b>目前可蒐集到的</b>路段流率資料。所以，是
#### 試着填補其他路段的流量狀況，回推整個系統的流動分佈的過程。  

### 以瞭解一個 城市的日常問題

### 連接性與設計 ( Design )
- 街道的結構是否讓某些路段 <b>註定</b> 會容易擁塞  
- 路網結構的問題

### 如何應用網路分析的方法，捕捉街道路網的連接性與設計的特性，作爲分析街道的基礎？

### 潛在需求與密度 ( Density )
- 城市的佈局(layout)
- 各個區域的功能
- 人口密度

### 在現有的路網結構下，以及在目前的流動需求下，車子會聚集在哪些區域？ -- 潛在需求

### 複雜度與多樣性 ( Diversity )
- 車子轉向的狀況與移動的關係 ?
- 附件建成環境的特性， 可及功能 (Access function) vs. 流動功能 (movement function) ?
- 附近其他交通工具的影響？

### 車子的轉向狀況對移動速度的影響與原因爲何？

## 下一步
- 進一步改善分析方法以加入其他特性如
    - 如高架道路、隧道...
    - damping factor (捕捉車子進入或離開街道系統的因子)
    - 移動方向及容量 (雙向道路及車道數)
- 分析複雜度、流率 與周邊建成環境之間的關係
    - 可及功能及流動功能
    - 複雜度與速度
    - 公共交通與大衆運輸

## 以上，感謝聆聽。


### 關於我: 
陳威全,  
臺大地理系的一個博士班學生。
wcchin.88@gmail.com  
http://wcchin.github.io/

links:  
- simulation video 1 - random turn: https://vimeo.com/168713881
- simulation video 2 - FBPR turn: https://vimeo.com/168714357
- 於發展年會研討會用的簡報: http://bit.ly/acds2016chin
